In [1]:
from pathlib import Path
import asyncio
import json
import fsspec
from collections import defaultdict

from kerchunk.combine import MultiZarrToZarr, merge_vars, auto_dask
from kerchunk.zarr import ZarrToZarr

import numpy as np
import xarray as xr
import pvdeg

from dask.distributed import LocalCluster, Client

MODEL_OUTS_DIR = Path("/projects/inspire/PySAM-MAPS/v1.1/model-outs/")
POSTPROCESS_OUTS_DIR = Path("/projects/inspire/PySAM-MAPS/v1.1/postprocess/")

Nodebook to test combining zarrs. Dask combined script moved to inspire_agrivolt_package/inspire_agrivolt/scripts/combine.py

In [40]:
client = Client(
    LocalCluster(
        n_workers=32
    )
)

print(client.dashboard_link)

http://127.0.0.1:8787/status


In [14]:
def load_model_outs_zarrs(confs: list[str]) -> dict[str, list[xr.Dataset]]: 
    """
    Find and load all model outputs results zarrs.
    """
    # FIND all MODEL_OUTS paths by config
    model_outs_all_conf_zarrs_paths = {}
    for conf in confs:
        conf_zarrs_paths = model_outs_all_conf_zarrs_paths.get(conf, [])
        model_outs_all_conf_zarrs_paths[conf] = conf_zarrs_paths

        for dir in MODEL_OUTS_DIR.iterdir():
            for conf_dir in dir.glob(conf):
                model_outs_all_conf_zarrs_paths[conf] += list(conf_dir.glob("*.zarr"))
        print(f"found {len(model_outs_all_conf_zarrs_paths[conf])} zarrs for model outs conf {conf}")

    # LOAD all MODEL_OUTS zarrs by config
    model_outs_all_conf_zarrs = {}
    for conf in confs:
        print(f"loading model outs zarrs for conf {conf}")
        conf_zarrs = model_outs_all_conf_zarrs.get(conf, [])
        model_outs_all_conf_zarrs[conf] = conf_zarrs

        for path in model_outs_all_conf_zarrs_paths[conf]:
            model_outs_chunk = xr.open_zarr(path)
            model_outs_all_conf_zarrs[conf].append(model_outs_chunk)
        print(f"loaded {len(model_outs_all_conf_zarrs[conf])} MODEL OUTS zarrs to for config {conf}.")

    return model_outs_all_conf_zarrs

def load_postprocessing_zarrs(confs: list[str]) -> dict[str, list[xr.Dataset]]: 
    """
    Find and load all postprocessing results zarrs.
    """
    # find all postprocessing zarrs paths
    postprocess_all_conf_zarrs_paths = {}
    for conf in confs:
        conf_zarrs_paths = postprocess_all_conf_zarrs_paths.get(conf, [])
        postprocess_all_conf_zarrs_paths[conf] = conf_zarrs_paths

        for state_dir in POSTPROCESS_OUTS_DIR.iterdir():
            for conf_zarr in state_dir.glob(f"{conf}.zarr"):

                postprocess_all_conf_zarrs_paths[conf].append(conf_zarr)

    # load all postprocessing zarrs
    postprocess_all_conf_zarrs = {}
    for conf in confs:
        print(f"loading postprocessing zarrs for conf {conf}")
        conf_zarrs = postprocess_all_conf_zarrs.get(conf, [])
        postprocess_all_conf_zarrs[conf] = conf_zarrs

        for path in postprocess_all_conf_zarrs_paths[conf]:
            postprocess_state_conf = xr.open_zarr(path).drop_dims(10) # extra dim in dataset
            postprocess_all_conf_zarrs[conf].append(postprocess_state_conf)

        print(f"loaded {len(postprocess_all_conf_zarrs[conf])} POSTPROCESS zarrs to for config {conf}.")

    return postprocess_all_conf_zarrs

def ground_irradiance_distances(ds: xr.Dataset) -> xr.DataArray:
    """
    Calculate beds distances from pitch.
    """
    NUM_BEDS = 10
    pitch = ds.pitch

    frac = xr.DataArray(
        (np.arange(NUM_BEDS) + 0.5) / NUM_BEDS,
        dims=(NUM_BEDS,),
        name=f"{NUM_BEDS}_fraction",
    )

    distances = (pitch * frac).rename("distances_m")
    return distances

In [15]:
confs = ["01"]

model_outs_all_conf_zarrs = load_model_outs_zarrs(confs)
postprocess_all_conf_zarrs = load_postprocessing_zarrs(confs)

model_res = xr.concat(model_outs_all_conf_zarrs['01'], dim="gid")
postprocessing_res = xr.concat(postprocess_all_conf_zarrs['01'], dim="gid")
combined = xr.merge([model_res, postprocessing_res])

found 1660 zarrs for model outs conf 01
loading model outs zarrs for conf 01
loaded 1660 MODEL OUTS zarrs to for config 01.
loading postprocessing zarrs for conf 01
loaded 49 POSTPROCESS zarrs to for config 01.


In [18]:
combined

<xarray.Dataset> Size: 1TB
Dimensions:                                (distance: 10, gid: 523385,
                                            time: 8760)
Coordinates:
  * distance                               (distance) int64 80B 0 1 2 ... 7 8 9
  * gid                                    (gid) int64 4MB 18316 ... 1465865
  * time                                   (time) datetime64[ns] 70kB 2001-01...
Data variables: (12/28)
    albedo                                 (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    annual_energy                          (gid) float64 4MB dask.array<chunksize=(39,), meta=np.ndarray>
    annual_poa                             (gid) float64 4MB dask.array<chunksize=(39,), meta=np.ndarray>
    dhi                                    (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    dni                                    (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    ghi                                    (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    ...                                     ...
    edgetoedge                             (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    edgetoedge_par                         (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    farmable_land_percent                  (gid) float64 4MB dask.array<chunksize=(38,), meta=np.ndarray>
    subarray1_dc_gross_per_acre            (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    subarray1_dc_gross_per_kWdc_installed  (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    underpanel                             (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
Attributes:
    kestrel_nsrdb_fnames:  ['/datasets/NSRDB/current/nsrdb_tmy-2024.h5']
    software_versions:     {'dask': '2024.8.0', 'dask_jobqueue': '0.8.5', 'fs...

In [37]:
def ground_irradiance_distances(ds: xr.Dataset) -> xr.DataArray:
    """
    Calculate beds distances from pitch.
    """
    NUM_BEDS = 10
    pitch = ds.pitch

    frac = xr.DataArray(
        (np.arange(NUM_BEDS) + 0.5) / NUM_BEDS,
        dims=(NUM_BEDS,),
        name=f"{NUM_BEDS}_fraction",
    )

    distances = (pitch * frac).rename("distances_m")
    return distances

if "distances_m" not in combined.data_vars:
    print("distances_m not in combined result, calculating beds distances from pitch")
    distances_m_da = ground_irradiance_distances(combined)

combined = xr.merge([combined, distances_m_da])

distances_m not in combined result, calculating beds distances from pitch


In [36]:
rename = {}
for data_var in combined.data_vars:
    if data_var.startswith("subarray1_"):
        rename[data_var] = data_var.lstrip("subarray1_")


if "under_panel" in combined.data_vars:
    rename = rename | {"under_panel":"underpanel"}
combined = combined.rename(rename)

combined['distance'] = combined.distance.astype(np.int32)
combined['gid'] = combined.gid.astype(np.int32)

combined

<xarray.Dataset> Size: 1TB
Dimensions:                           (distance: 10, gid: 523385, time: 8760)
Coordinates:
  * distance                          (distance) int32 40B 0 1 2 3 4 5 6 7 8 9
  * gid                               (gid) int32 2MB 18316 18317 ... 1465865
  * time                              (time) datetime64[ns] 70kB 2001-01-01 ....
Data variables: (12/28)
    albedo                            (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    annual_energy                     (gid) float64 4MB dask.array<chunksize=(39,), meta=np.ndarray>
    annual_poa                        (gid) float64 4MB dask.array<chunksize=(39,), meta=np.ndarray>
    dhi                               (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    dni                               (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    ghi                               (gid, time) float64 37GB dask.array<chunksize=(39, 8760), meta=np.ndarray>
    ...                                ...
    edgetoedge                        (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    edgetoedge_par                    (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    farmable_land_percent             (gid) float64 4MB dask.array<chunksize=(38,), meta=np.ndarray>
    dc_gross_per_acre                 (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    dc_gross_per_kWdc_installed       (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
    underpanel                        (gid, time) float64 37GB dask.array<chunksize=(38, 8760), meta=np.ndarray>
Attributes:
    kestrel_nsrdb_fnames:  ['/datasets/NSRDB/current/nsrdb_tmy-2024.h5']
    software_versions:     {'dask': '2024.8.0', 'dask_jobqueue': '0.8.5', 'fs...

In [ ]:
store_path = Path("/projects/inspire/PySAM-MAPS/v1.1/final/01.zarr")

if not store_path.exists():
    # triggers computation, none has been done up to this point

    if 10 in combined.dims:
        combined = combined.drop_dims([10])

    # print rechunking
    combined = combined.chunk({"gid":40, "distance":-1, "time":-1})
    combined = combined.unify_chunks()

    # avoid rechunk during write
    encoding = {
        v: {"chunks": tuple(chunk_sizes) for chunk_sizes in combined[v].chunks}
        for v in combined.data_vars
    }
    
    print("writing to path")
    combined.to_zarr(
        store_path, 
        compute=True,
        consolidated=True,
        encoding=encoding
    )

writing to path


/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 439.43 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
